## Display & Joystick의 객체화 및 캐릭터 조종

필요한 library import

In [1]:
import time
import random
from colorsys import hsv_to_rgb
import board
from digitalio import DigitalInOut, Direction
from PIL import Image, ImageDraw, ImageFont
from adafruit_rgb_display import st7789

Joystick이라는 클래스를 만들어, 간단한 모듈화 진행

In [2]:
class Joystick:
    def __init__(self):
        self.cs_pin = DigitalInOut(board.CE0)
        self.dc_pin = DigitalInOut(board.D25)
        self.reset_pin = DigitalInOut(board.D24)
        self.BAUDRATE = 24000000

        self.spi = board.SPI()
        self.disp = st7789.ST7789(
                    self.spi,
                    height=240,
                    y_offset=80,
                    rotation=180,
                    cs=self.cs_pin,
                    dc=self.dc_pin,
                    rst=self.reset_pin,
                    baudrate=self.BAUDRATE,
                    )

        # Input pins:
        self.button_A = DigitalInOut(board.D5)
        self.button_A.direction = Direction.INPUT

        self.button_B = DigitalInOut(board.D6)
        self.button_B.direction = Direction.INPUT

        self.button_L = DigitalInOut(board.D27)
        self.button_L.direction = Direction.INPUT

        self.button_R = DigitalInOut(board.D23)
        self.button_R.direction = Direction.INPUT

        self.button_U = DigitalInOut(board.D17)
        self.button_U.direction = Direction.INPUT

        self.button_D = DigitalInOut(board.D22)
        self.button_D.direction = Direction.INPUT

        self.button_C = DigitalInOut(board.D4)
        self.button_C.direction = Direction.INPUT

        # Turn on the Backlight
        self.backlight = DigitalInOut(board.D26)
        self.backlight.switch_to_output()
        self.backlight.value = True

        # Create blank image for drawing.
        # Make sure to create image with mode 'RGB' for color.
        self.width = self.disp.width
        self.height = self.disp.height

joystick 객체 선언

In [ ]:
joystick = Joystick()

## Display와 친해지기

display 초기화
- PIL에서 지원하는 Image를 사용, joystick의 width, height에 맞는 Image 생성 (하나의 도화지를 만들었다고 생각)

In [4]:
my_image = Image.new("RGB", (joystick.width, joystick.height))

만들어진 도화지에 그림을 그리기 위한 도구 선언
- ImageDraw의 Draw함수를 통해 우리가 선언했던 my_image에 그림을 그리는 도구인 my_draw를 선언

In [5]:
my_draw = ImageDraw.Draw(my_image)

선언된 my_draw를 이용해 rectangle을 도화지에 그려보기
- rectangle((사각형의 왼쪽 위 꼭짓점 위치x1 y1, 사각형의 오른쪽 아래 꼭짓점 위치x2 y2)
, fill = (R, G, B, A))
- A는 알파 값으로 0~ 100의 값을 갖고, 투명하게 만들건지 아닌지를 결정함 (0 -> 투명 ~ 100 -> 불투명)

In [6]:
my_draw.rectangle((0, 0, joystick.width, joystick.height), fill=(255, 0, 0, 100))

빨간색 rectangle을 (0, 0)에서부터 width, height를 꽉 차도록 그린 도화지를 display에 띄우기
- 꼭 fill을 (255, 0, 0, 100) 으로 하지 않아도 되고, 원하는 색으로 해보세요!

In [7]:
joystick.disp.image(my_image)

캐릭터를 만들기 전, 하얀색 배경을 만들고 화면에 띄워보고, 캐릭터가 될 동그라미 그려보기

In [8]:
my_draw.rectangle((0, 0, joystick.width, joystick.height), fill = (255, 255, 255, 100))
my_draw.ellipse((100, 50, 140, 90), outline = "#FFFFFF", fill = (0, 0, 0))
#좌표는 동그라미의 왼쪽 위, 오른쪽 아래 점 (x1, y1, x2, y2)
joystick.disp.image(my_image)

## 캐릭터 만들기

간단한 모양(네모, 동그라미, ..)을 가지고 조종할 수 있는 캐릭터를 만들어보기
- 기능 -> 움직이기, 움직일때는 빨간 테두리를, 가만히 있을 때는 하얀 테두리를 가지게 할 것

In [9]:
import numpy as np
class Character:
    def __init__(self, width, height):
        self.appearance = 'circle'
        self.state = None
        self.position = np.array([width/2 - 20, height/2 - 20, width/2 + 20, height/2 + 20])
        self.outline = "#FFFFFF"

    def move(self, command = None):
        if command == None:
            self.state = None
            self.outline = "#FFFFFF" #검정색상 코드!
        
        else:
            self.state = 'move'
            self.outline = "#FF0000" #빨강색상 코드!

            if command == 'up_pressed':
                self.position[1] -= 5
                self.position[3] -= 5

            elif command == 'down_pressed':
                self.position[1] += 5
                self.position[3] += 5

            elif command == 'left_pressed':
                self.position[0] -= 5
                self.position[2] -= 5
                
            elif command == 'right_pressed':
                self.position[0] += 5
                self.position[2] += 5

In [ ]:
my_circle = Character(joystick.width, joystick.height)
my_draw.rectangle((0, 0, joystick.width, joystick.height), fill = (255, 255, 255, 100))
while True:
    command = None
    if not joystick.button_U.value:  # up pressed
        command = 'up_pressed'

    elif not joystick.button_D.value:  # down pressed
        command = 'down_pressed'

    elif not joystick.button_L.value:  # left pressed
        command = 'left_pressed'

    elif not joystick.button_R.value:  # right pressed
        command = 'right_pressed'
        
    else:
        command = None

    my_circle.move(command)

    my_draw.ellipse(tuple(my_circle.position), outline = my_circle.outline, fill = (0, 0, 0))
#좌표는 동그라미의 왼쪽 위, 오른쪽 아래 점 (x1, y1, x2, y2)
    joystick.disp.image(my_image)

실행시켜보면 문제점이 무엇인가요?
- 캐릭터가 움직이면서 잔상같은게 남습니다. 이유가 무엇인가요?
- 이전에 그려졌던 circle이 사라지지 않음  
-> 해결 방법 : 하얀색 도화지를 새로 깐다, 이전위치에 배경색과 똑같은 동그라미를 다시 그린다, 등등 ..

In [ ]:
# 잔상이 남지 않는 코드
my_circle = Character(joystick.width, joystick.height)
my_draw.rectangle((0, 0, joystick.width, joystick.height), fill = (255, 255, 255, 100))
while True:
    command = None
    if not joystick.button_U.value:  # up pressed
        command = 'up_pressed'

    elif not joystick.button_D.value:  # down pressed
        command = 'down_pressed'

    elif not joystick.button_L.value:  # left pressed
        command = 'left_pressed'

    elif not joystick.button_R.value:  # right pressed
        command = 'right_pressed'
        
    else:
        command = None

    my_circle.move(command)

    #그리는 순서가 중요합니다. 배경을 먼저 깔고 위에 그림을 그리고 싶었는데 그림을 그려놓고 배경으로 덮는 결과로 될 수 있습니다.
    my_draw.rectangle((0, 0, joystick.width, joystick.height), fill = (255, 255, 255, 100))
    my_draw.ellipse(tuple(my_circle.position), outline = my_circle.outline, fill = (0, 0, 0))
#좌표는 동그라미의 왼쪽 위, 오른쪽 아래 점 (x1, y1, x2, y2)
    joystick.disp.image(my_image)

또다른 문제는 무엇인가요?
- 캐릭터가 대각선으로 움직이지 않습니다  
-> 문제가 일어나는 곳을 파악하고, 스스로 해결해보세요😃

jupyter notebook을 사용해서 돌려도 보고, .py 파일로 만들어 실행시켜보세요!